In [1]:
data = open(
    '/content/drive/MyDrive/Colab Notebooks/AOC/2022/day14.txt',
    'r').read().split("\n")

In [2]:
import numpy as np
from functools import reduce
from operator import itemgetter
np.set_printoptions(linewidth=np.inf)

def make_grid(input):
    rocks = []  # all rock positions
    def generatePath(from_, to_):
        '''Generates all intermediate coordinates between each pair of points
        '''
        # vertical path
        if from_[0] == to_[0]:  # x1 = x2
            if from_[1] < to_[1]: # y1 < y2
                y_values = list(range(from_[1]+1, to_[1]+1))
            else: # y1 >= y2
                y_values = list(range(from_[1]-1, to_[1]-1, -1))
            for y in y_values:
                rocks.append((from_[0], y))
        # horizontal path       
        else: # y1 = y2
            if from_[0] < to_[0]: # x1 < x2
                x_values = list(range(from_[0]+1, to_[0]+1))
            else: # x1 >= x2
                x_values = list(range(from_[0]-1, to_[0]-1, -1))
            for x in x_values:
                rocks.append((x, from_[1]))
        
        return to_

    for path in input:
        coords = [tuple(map(int,x.split(","))) for x in path.split(" -> ")]
        rocks.append(coords[0])
        reduce(lambda x, y: generatePath(x, y), coords)

    rocks.sort(key = itemgetter(0))
    XMIN, XMAX = rocks[0][0], rocks[-1][0]  # set horizontal width of grid
    YMAX = max(rocks, key = itemgetter(1))[1]
    print(XMIN, XMAX, YMAX)

    cave = np.array([["." for i in range(XMIN, XMAX + 1)] for j in range(YMAX + 1)])
    
    for col,row in rocks:
        cave[row, (col-XMIN) % (XMAX - XMIN + 1)] = "#"

    return cave, (XMIN, XMAX)

In [3]:
def simulate(cave, source_x, source_y):
    m, n = cave.shape[0], cave.shape[1]
    def move_sand(row, col):
        r_temp = None
        # Try to move downwards
        for r in range(row, m-1):
            # next position down is occupied
            if cave[r + 1, col] == "#" or cave[r + 1, col] == "o":
                r_temp = r
                break

        if r_temp:
            # Try to move down left or right
            for r,c in [(r_temp + 1, col - 1), (r_temp + 1, col + 1)]:
                if c < 0 or c >= n: # off the grid
                    return False
                elif cave[r, c] == ".":
                    return move_sand(r,c)

            cave[r_temp, col] = "o"
            return True
        
        return False

    ans = 0
    while True:
        atRest = move_sand(source_y, source_x)
        if atRest: ans += 1
        else: break

    return ans

In [4]:
cave, xbounds = make_grid(data)
source_x = (500 - xbounds[0]) % (xbounds[1] - xbounds[0] + 1)
simulate(cave, source_x, 0)

479 548 174


979

In [5]:
def simulateP2(input):
    obs = set()
    ans = 0
    def generatePath(from_, to_):
        '''Generates all intermediate coordinates between each pair of points
        '''
        if from_[0] == to_[0]:  # x1 = x2
            if from_[1] < to_[1]: # y1 < y2
                y_values = list(range(from_[1]+1, to_[1]+1))
            else: # y1 >= y2
                y_values = list(range(from_[1]-1, to_[1]-1, -1))
            for y in y_values:
                obs.add((from_[0], y))      
        else: # y1 = y2
            if from_[0] < to_[0]: # x1 < x2
                x_values = list(range(from_[0]+1, to_[0]+1))
            else: # x1 >= x2
                x_values = list(range(from_[0]-1, to_[0]-1, -1))
            for x in x_values:
                obs.add((x, from_[1]))
        
        return to_

    def move_sand():
        x, y = 500, 0
        while True:
            if (500, 0) in obs:
                return True
            elif (x, y+1) not in obs and y+1 < floor:
                y = y+1
            elif (x-1, y+1) not in obs and y+1 < floor:
                x,y = x-1, y+1
            elif (x+1, y+1) not in obs and y+1 < floor:
                x,y = x+1, y+1
            else:
                obs.add((x, y))
                return False

    for path in input:
        coords = [tuple(map(int,x.split(","))) for x in path.split(" -> ")]
        obs.add(coords[0])
        reduce(lambda x, y: generatePath(x, y), coords)

    floor = max(obs, key = itemgetter(1))[1] + 2

    while True:
        isBlocked = move_sand()
        ans += 1
        if isBlocked: break

    return ans-1

In [6]:
simulateP2(data)

29044